Sumit Parwal   INF 552 Q1 5174593050

In [2]:
from __future__ import print_function
from keras.callbacks import LambdaCallback,ModelCheckpoint
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
from keras.preprocessing.text import one_hot
import keras
import numpy as np
import random
import sys
import io,os
import re

from __future__ import absolute_import, division, unicode_literals

import tensorflow as tf
from tensorflow import keras

tf.__version__

Using TensorFlow backend.


'1.14.0'

In [3]:
!pip install h5py pyyaml
!pip install tf_nightly

     |████████████████████████████████| 102.1MB 1.2MB/s 
     |████████████████████████████████| 501kB 45.1MB/s 
     |████████████████████████████████| 61kB 27.0MB/s 
     |████████████████████████████████| 4.1MB 40.7MB/s 
  Stored in directory: /root/.cache/pip/wheels/51/3e/a3/b351fae0cbf15373c2136a54a70f43fea5fe91d8168a5faaa4
Successfully built opt-einsum


b)

In [4]:
from google.colab import files
uploaded = files.upload()

Saving AIIMAT.txt to AIIMAT.txt
Saving MLOE.txt to MLOE.txt
Saving OKEWFSMP.txt to OKEWFSMP.txt
Saving TAM.txt to TAM.txt
Saving TAMatter.txt to TAMatter.txt
Saving THWP.txt to THWP.txt
Saving TPP.txt to TPP.txt


In [0]:
import io
text0 =  io.BytesIO(uploaded['AIIMAT.txt']).read().lower().decode('ISO-8859-1')
text1 =  io.BytesIO(uploaded['MLOE.txt']).read().lower().decode('ISO-8859-1')
text2 =  io.BytesIO(uploaded['OKEWFSMP.txt']).read().lower().decode('ISO-8859-1')
text3 =  io.BytesIO(uploaded['TAM.txt']).read().lower().decode('ISO-8859-1')
text4 =  io.BytesIO(uploaded['TAMatter.txt']).read().lower().decode('ISO-8859-1')
text5 =  io.BytesIO(uploaded['THWP.txt']).read().lower().decode('ISO-8859-1')
text6 =  io.BytesIO(uploaded['TPP.txt']).read().lower().decode('ISO-8859-1')

# len(text6)

c)i)

In [0]:
text=text1+text2+text3+text5+text6

In [7]:
len(text)

3640664

In [0]:
text = re.sub('[^a-zA-Z\.\,]',' ', text)

In [0]:
text=re.sub( '\s+', ' ', text ).strip()

In [0]:
alpharray=list(text)

In [0]:
array_ascii=[ord(c) for c in alpharray]
array_scaled=[((c-32)/90) for c in array_ascii]

In [0]:
to_char=dict((c,chr(int((c*90)+32))) for i, c in enumerate(set(array_scaled)))
to_ascii=dict((chr(int((c*90)+32)),c) for i, c in enumerate(set(array_scaled)))
del alpharray

In [13]:
to_ascii

{' ': 0.0,
 ',': 0.13333333333333333,
 '.': 0.15555555555555556,
 'a': 0.7222222222222222,
 'b': 0.7333333333333333,
 'c': 0.7444444444444445,
 'd': 0.7555555555555555,
 'e': 0.7666666666666667,
 'f': 0.7777777777777778,
 'g': 0.7888888888888889,
 'h': 0.8,
 'i': 0.8111111111111111,
 'j': 0.8222222222222222,
 'k': 0.8333333333333334,
 'l': 0.8444444444444444,
 'm': 0.8555555555555555,
 'n': 0.8666666666666667,
 'o': 0.8777777777777778,
 'p': 0.8888888888888888,
 'q': 0.9,
 'r': 0.9111111111111111,
 's': 0.9222222222222223,
 't': 0.9333333333333333,
 'u': 0.9444444444444444,
 'v': 0.9555555555555556,
 'w': 0.9666666666666667,
 'x': 0.9777777777777777,
 'y': 0.9888888888888889,
 'z': 1.0}

In [0]:
to_enc={}
inc=0
for c in sorted(to_ascii.values()):
    to_enc.update({c:inc})
    inc+=1

In [0]:
decode={}
inc=0
for c in sorted(to_ascii.values()):
    decode.update({inc:c})
    inc+=1

In [0]:
chars=set(array_scaled)


iii)Choose a window size, e.g., W = 100.

iv) Inputs to the network will be the first W􀀀1 = 99 characters of each sequence,
and the output of the network will be the Wth character of the sequence.
Basically, we are training the network to predict each character using the 99
characters that precede it. Slide the window in strides of S = 1 on the text.
For example, if W = 5 and S = 1 and we want to train the network with the
sequence ABRACADABRA, The first input to the network will be ABRA
and the corresponding output will be C. The second input will be BRAC and
the second output will be A, etc.

v) Note that the output has to be encoded using a one-hot encoding scheme with
N = 256 (or less) elements. This means that the network reads integers, but
outputs a vector of N = 256 (or less) elements.

vi) Use a single hidden layer for the LSTM with N = 256 (or less) memory units.

vii) Use a Softmax output layer to yield a probability prediction for each of the
characters between 0 and 1. This is actually a character classication problem
with N classes. Choose log loss (cross entropy) as the objective function for
the network (research what it means).3

viii)We do not use a test dataset. We are using the whole training dataset to
learn the probability of each character in a sequence. We are not seeking for
a very accurate model. Instead we are interested in a generalization of the
dataset that can mimic the gist of the text.

ix.) Choose a reasonable number of epochs4 for training, considering your compu-
tational power (e.g., 30, although the network will need more epochs to yield
a better model).

x.) Use model checkpointing to keep the network weights to determine each time
an improvement in loss is observed at the end of the epoch. Find the best set
of weights in terms of loss.

xi) Use the network with the best weights to generate 1000 characters, using the
following text as initialization of the network:
There are those who take mental phenomena naively, just as they
would physical phenomena. This school of psychologists tends not to
emphasize the object.

In [17]:
N = 99

def nSentences(fullarray,maxlen):
    step = 1
    sentences = []
    next_chars = []
    for i in range(0, len(fullarray) - maxlen, step):
        sentences.append(fullarray[i: i + maxlen])
        next_chars.append(fullarray[i + maxlen])
    print('nb sequences:', len(sentences))
    return sentences,next_chars
sentences,next_char=nSentences(array_scaled,N)
del array_scaled

nb sequences: 3510278


In [18]:
print('Produce X and Y to train')
x = np.zeros((len(sentences), N, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
     for k , char in enumerate(sentence):
        x[i, k , toenc[char]] = 1
        y[i, toenc[next_char[i]]] = 1
x.shape

Produce X and Y to train


(3510278, 99, 29)

In [0]:
del sentences

In [20]:
model = Sequential()
model.add(LSTM(256, input_shape=(N, len(chars))))
model.add(Dropout(0.4))
model.add(Dense(len(chars),activation='softmax'))
model.summary()

W0723 17:24:39.560538 140415099955072 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0723 17:24:39.607721 140415099955072 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0723 17:24:39.614494 140415099955072 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0723 17:24:39.948050 140415099955072 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0723 17:24:39.955442 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 256)               292864    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 29)                7453      
Total params: 300,317
Trainable params: 300,317
Non-trainable params: 0
_________________________________________________________________




x. Use model checkpointing to keep the network weights to determine each time an improvement in loss is observed at the end of the epoch. Find the best set of weights in terms of loss. 


In [0]:
# define the checkpoint
checkpoint_path ="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint_dir = os.path.dirname(checkpoint_path)
checkpoint = ModelCheckpoint(checkpoint_path, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [22]:
#optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='categorical_crossentropy', optimizer='adam')

W0723 17:24:39.995512 140415099955072 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0723 17:24:40.018582 140415099955072 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [23]:
model.fit(x,y, epochs=50, batch_size=5000,callbacks=callbacks_list)

W0723 17:24:40.144826 140415099955072 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/50
3510278/3510278 [==============================] - 440s 125us/step - loss: 2.2435

Epoch 00001: loss improved from inf to 2.24349, saving model to weights-improvement-01-2.2435.hdf5
Epoch 2/50
3510278/3510278 [==============================] - 444s 127us/step - loss: 1.7685

Epoch 00002: loss improved from 2.24349 to 1.76851, saving model to weights-improvement-02-1.7685.hdf5
Epoch 3/50
3510278/3510278 [==============================] - 445s 127us/step - loss: 1.5822

Epoch 00003: loss improved from 1.76851 to 1.58221, saving model to weights-improvement-03-1.5822.hdf5
Epoch 4/50
3510278/3510278 [==============================] - 446s 127us/step - loss: 1.4766

Epoch 00004: loss improved from 1.58221 to 1.47661, saving model to weights-improvement-04-1.4766.hdf5
Epoch 5/50
3510278/3510278 [==============================] - 446s 127us/step - loss: 1.4091

Epoch 00005: loss improved from 1.47661 to 1.40910, saving model to weights-improvement-05-1.4091.hdf5
Epoch 6/50
3510278/

In [24]:
ques="There are those who take mental phenomena naively, just as they would physical phenomena. This school of psychologists tends not to emphasize the object."
ques=ques.lower()
ques=ques[-N:]
len(ques)

99

In [0]:
test_array_ascii=[ord(c) for c in ques]
test_scaled_array=[((c-32)/90) for c in test_array_ascii]

In [0]:
test_sentences=[test_scaled_array]
x_t = np.zeros((len(test_sentences), N, len(chars)), dtype=np.bool)
for i, tsentence in enumerate(test_sentences):
     for k , char in enumerate(tsentence):
        x_t[i, k , toenc[char]] = 1

In [0]:
x_t.shape
prediction = model.predict(x_t, verbose=0)

In [28]:
print(ques)
print("Generated text :")
for i in range(1000):
    prediction = model.predict(x_t, verbose=0)
    print(tochar[decode[np.argmax(prediction)]],end="")
    x_t[0][:-1]=x_t[0][1:]
    k=np.zeros(len(chars), dtype=np.bool)
    k[np.argmax(prediction)]=1
    x_t[0][-1]=k

t as they would physical phenomena. this school of psychologists tends not to emphasize the object.
Generated text :
 the sense data is the same that it is the same that it is the same that there is no such thing as the sense data which is the same that it is the same that there is no such thing as the sense data which is the same that it is the same that there is no such thing as the sense data which is the same that it is the same that there is no such thing as the sense data which is the same that it is the same that there is no such thing as the sense data which is the same that it is the same that there is no such thing as the sense data which is the same that it is the same that there is no such thing as the sense data which is the same that it is the same that there is no such thing as the sense data which is the same that it is the same that there is no such thing as the sense data which is the same that it is the same that there is no such thing as the sense data which is the 

In [29]:
print(ques)
print("Generated text :")
for i in range(1000):
    prediction = model.predict(x_t, verbose=0)
    print(tochar[decode[np.argmax(prediction)]],end="")
    x_t[0][:-1]=x_t[0][1:]
    k=np.zeros(len(chars), dtype=np.bool)
    k[np.argmax(prediction)]=1
    x_t[0][-1]=k

t as they would physical phenomena. this school of psychologists tends not to emphasize the object.
Generated text :
re is no such thing as the sense data which is the same that it is the same that there is no such thing as the sense data which is the same that it is the same that there is no such thing as the sense data which is the same that it is the same that there is no such thing as the sense data which is the same that it is the same that there is no such thing as the sense data which is the same that it is the same that there is no such thing as the sense data which is the same that it is the same that there is no such thing as the sense data which is the same that it is the same that there is no such thing as the sense data which is the same that it is the same that there is no such thing as the sense data which is the same that it is the same that there is no such thing as the sense data which is the same that it is the same that there is no such thing as the sense data which

In [30]:
print(ques)
print("Generated text :")
for i in range(400):
    prediction = model.predict(x_t, verbose=0)
    print(tochar[decode[np.argmax(prediction)]],end="")
    x_t[0][:-1]=x_t[0][1:]
    k=np.zeros(len(chars), dtype=np.bool)
    k[np.argmax(prediction)]=1
    x_t[0][-1]=k

t as they would physical phenomena. this school of psychologists tends not to emphasize the object.
Generated text :
that there is no such thing as the sense data which is the same that it is the same that there is no such thing as the sense data which is the same that it is the same that there is no such thing as the sense data which is the same that it is the same that there is no such thing as the sense data which is the same that it is the same that there is no such thing as the sense data which is the same 

In [31]:
print(ques)
print("Generated text :")
for i in range(1000):
    prediction = model.predict(x_t, verbose=0)
    print(tochar[decode[np.argmax(prediction)]],end="")
    x_t[0][:-1]=x_t[0][1:]
    k=np.zeros(len(chars), dtype=np.bool)
    k[np.argmax(prediction)]=1
    x_t[0][-1]=k

t as they would physical phenomena. this school of psychologists tends not to emphasize the object.
Generated text :
that it is the same that there is no such thing as the sense data which is the same that it is the same that there is no such thing as the sense data which is the same that it is the same that there is no such thing as the sense data which is the same that it is the same that there is no such thing as the sense data which is the same that it is the same that there is no such thing as the sense data which is the same that it is the same that there is no such thing as the sense data which is the same that it is the same that there is no such thing as the sense data which is the same that it is the same that there is no such thing as the sense data which is the same that it is the same that there is no such thing as the sense data which is the same that it is the same that there is no such thing as the sense data which is the same that it is the same that there is no such t

In [32]:
print(ques)
print("Generated text :")
for i in range(1000):
    prediction = model.predict(x_t, verbose=0)
    print(tochar[decode[np.argmax(prediction)]],end="")
    x_t[0][:-1]=x_t[0][1:]
    k=np.zeros(len(chars), dtype=np.bool)
    k[np.argmax(prediction)]=1
    x_t[0][-1]=k

t as they would physical phenomena. this school of psychologists tends not to emphasize the object.
Generated text :
he same that it is the same that there is no such thing as the sense data which is the same that it is the same that there is no such thing as the sense data which is the same that it is the same that there is no such thing as the sense data which is the same that it is the same that there is no such thing as the sense data which is the same that it is the same that there is no such thing as the sense data which is the same that it is the same that there is no such thing as the sense data which is the same that it is the same that there is no such thing as the sense data which is the same that it is the same that there is no such thing as the sense data which is the same that it is the same that there is no such thing as the sense data which is the same that it is the same that there is no such thing as the sense data which is the same that it is the same that there is n

In [33]:
! ls {checkpoint_dir}

AIIMAT.txt			    weights-improvement-22-1.1621.hdf5
MLOE.txt			    weights-improvement-23-1.1582.hdf5
OKEWFSMP.txt			    weights-improvement-24-1.1543.hdf5
sample_data			    weights-improvement-25-1.1507.hdf5
TAMatter.txt			    weights-improvement-26-1.1471.hdf5
TAM.txt				    weights-improvement-27-1.1437.hdf5
THWP.txt			    weights-improvement-28-1.1410.hdf5
TPP.txt				    weights-improvement-29-1.1378.hdf5
weights-improvement-01-2.2435.hdf5  weights-improvement-30-1.1355.hdf5
weights-improvement-02-1.7685.hdf5  weights-improvement-31-1.1329.hdf5
weights-improvement-03-1.5822.hdf5  weights-improvement-32-1.1307.hdf5
weights-improvement-04-1.4766.hdf5  weights-improvement-33-1.1282.hdf5
weights-improvement-05-1.4091.hdf5  weights-improvement-34-1.1263.hdf5
weights-improvement-06-1.3620.hdf5  weights-improvement-35-1.1244.hdf5
weights-improvement-07-1.3271.hdf5  weights-improvement-36-1.1223.hdf5
weights-improvement-08-1.3003.hdf5  weights-improvement-37-1.1201.hdf5
weights-improvement

In [0]:
#Load backed up model
filename = "weights-improvement-21-1.1667.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')